# Milvus DB

## Initialie a connection to Milvus

First you need to pull and run a Milvus docker image: 
```
docker pull ...
```

Then we define the 


In [1]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    MilvusClient
)
import numpy as np

import time

In [7]:
# To je samo zato da lahko razdelimo kodo v vec celic v notebooku
def add_method(cls):  
    def decorator(func):  
        setattr(cls, func.__name__, func)  
        return func  
    return decorator  

class SimpleVectorDB:
    def __init__(
        self, host="localhost", port="19530", vector_dim = 128, collection_name="simple_vectors", index_params={
                "metric_type": "COSINE",
                "index_type": "IVF_FLAT",
                "params": {"nlist": 1024},
            }
    ):
        """Initialize connection to Milvus server and set up the collection."""
        self.collection_name = collection_name

        # Connect to Milvus server
        connections.connect(host=host, port=port)

        # Define collection schema
        self.dim = vector_dim  # Vector dimension
        
        print("self.dim", self.dim)

        # Define the fields for the collection
        fields = [
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
            FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=self.dim),
            FieldSchema(name="metadata", dtype=DataType.VARCHAR, max_length=512),
        ]

        schema = CollectionSchema(fields=fields, description="Simple vector database")


        # Create collection if it doesn't exist
        if utility.has_collection(self.collection_name):
            self.collection = Collection(self.collection_name)
        else:
            self.collection = Collection(self.collection_name, schema)

            # Create an index for vector field
            self.collection.create_index(field_name="vector", index_params=index_params)





In [8]:
@add_method(SimpleVectorDB) 
def insert(self, vectors, metadata_list):
    """
    Insert vectors and their metadata into the database.

    Args:
        vectors: numpy array of shape (n, dim) containing the vectors
        metadata_list: list of strings containing metadata for each vector
    """
    if len(vectors) != len(metadata_list):
        raise ValueError("Number of vectors and metadata entries must match")

    # Ensure vectors are in the correct format (numpy array)
    if not isinstance(vectors, np.ndarray):
        vectors = np.array(vectors)

    # Ensure vectors are float32
    vectors = vectors.astype(np.float32)

    # Prepare data for insertion
    data = [
        {"vector": vec, "metadata": meta}
        for vec, meta in zip(vectors, metadata_list)
    ]

    # Insert the data
    self.collection.insert(data)
    self.collection.flush()


In [9]:
@add_method(SimpleVectorDB)
def search(self, query_vector, top_k=5, search_params={}):
    """
    Search for the closest vectors to the query vector.

    Args:
        query_vector: numpy array of shape (dim,) containing the query vector
        top_k: number of closest vectors to return

    Returns:
        List of tuples containing (id, distance, metadata) for the closest vectors
    """
    self.collection.load()

    # Ensure query vector is in the correct format
    if not isinstance(query_vector, np.ndarray):
        query_vector = np.array(query_vector)
    query_vector = query_vector.astype(np.float32)


    results = self.collection.search(
        data=[query_vector],
        anns_field="vector",
        param=search_params,
        limit=top_k,
        output_fields=["metadata"],
    )

    # Format results
    search_results = []
    for hits in results:
        for hit in hits:
            search_results.append(
                {
                    "id": hit.id,
                    "distance": hit.distance,
                    "metadata": hit.entity.get("metadata"),
                }
            )

    return search_results


In [10]:
@add_method(SimpleVectorDB)
def delete_by_ids(self, ids):
    """Delete vectors by their IDs."""
    expr = f"id in {ids}"
    self.collection.delete(expr)

@add_method(SimpleVectorDB)
def count(self):
    """Return the number of vectors in the database."""
    return self.collection.num_entities

@add_method(SimpleVectorDB)
def close(self):
    """Close the connection to Milvus server."""
    connections.disconnect("default")
    
@add_method(SimpleVectorDB)
def delete_collection(self):
    """Drop the collection"""
    self.collection.drop()


In [19]:

## FLAT Index

FLAT_INDEX_PARAMS = {
    "index_type": "FLAT",
    "metric_type": "COSINE",
}

VECTOR_DIM = 32

db = SimpleVectorDB(collection_name= "flat_index_collection", vector_dim=VECTOR_DIM, index_params=FLAT_INDEX_PARAMS)


def insertRandomVectors(db, num_vectors = 99999, vector_size = 128):
    # Create some random vectors and metadata
    vectors = (
        np.random.random((num_vectors, vector_size)).astype(np.float32) * 10000
    )  # Ensure float32 type
    # arr1, arr2, arr3 = np.split(vectors, 3)

    metadata = np.array([f"Vector_{i}" for i in range(num_vectors)]) 
    # mtd1, mtd2, mtd3 = np.split(metadata, 3)

    # Insert vectors
    db.insert(vectors, metadata)



t1 = time.time()    
for _ in range(3):
    insertRandomVectors(db, num_vectors=333333,  vector_size=VECTOR_DIM)
print(f"Inserted {3*333333} vectors in {time.time() - t1:.2f}s")

# Perform a search
t2 = time.time()
query = np.random.random(VECTOR_DIM).astype(np.float32).tolist()  # Ensure float32 type
for _ in range(10):
    results = db.search(query, top_k=3)
print(f"Search time: {time.time() - t2:.2f}s")

print("\nSearch results:", results)
for i in range(len(results)):
    print(f"ID: {results[i]['id']}, Distance: {results[i]['distance']}, Metadata: {results[i]['metadata']}")



db.delete_collection()




self.dim 32
[1320.7015 4334.335  8375.078  9053.737  9592.421  3919.1616 4671.051
 7666.9077 3783.1152 3940.2878 5429.492  3328.0615 4693.191  5740.129
 8452.602  4456.467   735.8236  815.8787 2449.722  9229.746  9520.31
 6514.063  7026.4263 1230.03    689.3257 8657.897  7663.351  6506.086
 1447.5658 3197.5156 3399.8633 9018.158 ]
[4138.949    5631.6255   5918.363    5325.3164   3415.9023   1286.868
   80.05101  7175.9854   3537.5234   3354.2332   8880.523     187.98361
 1970.9524   3317.586    1713.8632   5926.181    3103.514      13.513407
 5971.579    4570.1465    591.88495  5622.078    1746.9409   4520.202
 8843.883    1010.48627  4780.8896   6607.956    5414.582    8523.429
 6756.3184   3035.0671  ]
[2088.8699   3379.7837    755.73773  4729.734    3321.0974   9189.809
 6073.165      80.00221  2590.108    1889.8983   6212.091    9571.051
 3417.6802   3676.7861   3911.5532   8288.94     4508.042    6758.696
 8481.856      76.428024  982.20654  1098.8612   2363.8806   5982.217
 9048.

In [18]:
## HNSW Index

HNSW_INDEX_PARAMS = {
    "index_type": "HNSW",
    "metric_type": "COSINE",
    "params": {"M": 64, "efConstruction": 64},
}

db = SimpleVectorDB(collection_name= "hnsw_index_collection", vector_dim=VECTOR_DIM, index_params=HNSW_INDEX_PARAMS)


def insertRandomVectors(db, num_vectors = 99999, vector_size = 128):
    # Create some random vectors and metadata
    vectors = (
        np.random.random((num_vectors, vector_size)).astype(np.float32) * 10000
    )  # Ensure float32 type
    # arr1, arr2, arr3 = np.split(vectors, 3)

    metadata = np.array([f"Vector_{i}" for i in range(num_vectors)]) 
    # mtd1, mtd2, mtd3 = np.split(metadata, 3)

    # Insert vectors
    db.insert(vectors, metadata)



t1 = time.time()    
for _ in range(3):
    insertRandomVectors(db, num_vectors=333333,  vector_size=VECTOR_DIM)
print(f"Inserted {3*333333} vectors in {time.time() - t1:.2f}s")

# Perform a search
t2 = time.time()
query = np.random.random(VECTOR_DIM).astype(np.float32).tolist()  # Ensure float32 type
for _ in range(10):
    results = db.search(query, top_k=3)
print(f"Search time: {time.time() - t2:.2f}s")

print("\nSearch results:", results)
for i in range(len(results)):
    print(f"ID: {results[i]['id']}, Distance: {results[i]['distance']}, Metadata: {results[i]['metadata']}")



db.delete_collection()


self.dim 32
Inserted 999999 vectors in 54.83s
Search time: 5.16s

Search results: [{'id': 455257870267531501, 'distance': 0.957230269908905, 'metadata': 'Vector_302653'}, {'id': 455257870266780222, 'distance': 0.9456062316894531, 'metadata': 'Vector_84815'}, {'id': 455257870266822324, 'distance': 0.9422832727432251, 'metadata': 'Vector_126917'}]
ID: 455257870267531501, Distance: 0.957230269908905, Metadata: Vector_302653
ID: 455257870266780222, Distance: 0.9456062316894531, Metadata: Vector_84815
ID: 455257870266822324, Distance: 0.9422832727432251, Metadata: Vector_126917
